In [1]:
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np

mesh = ngs.Mesh('proximity_geo.vol')
mesh.Curve(3)
mesh.RefineHP(2,factor = 0.2)

In [2]:
# Draw(mesh)

In [3]:
# H1 = ngs.H1(mesh, order = 2, dirichlet = "ambient_face")
# cfJ = mesh.MaterialCF({"circle": 1}, default = 0)

# u,v = H1.TnT()

# K = ngs.BilinearForm(H1)
# K += (ngs.grad(u)*ngs.grad(v))*ngs.dx()
# K.Assemble()

# f = ngs.LinearForm(H1)
# f += (cfJ*v)*ngs.dx()
# f.Assemble()

# x = ngs.GridFunction(H1)
# x.vec.data += K.mat.Inverse(H1.FreeDofs())*f.vec

# # Draw(x)
# # Draw(cfJ,mesh)

In [4]:
# Parameters

circ_mass = np.pi*0.031**2
circ_number = 58

mu0 = 1.256*1e-6
mu_iron = 1*mu0
sigma = 58*1e6
J = 1e7/(circ_mass)

cf_Js = mesh.MaterialCF({"circle_left": J, "circle_right": -J}, default = 0)
cf_mu = mesh.MaterialCF({"circle_left|circle_right|left|right|bridge" : mu0, "ambient": mu_iron})
cf_sigma = mesh.MaterialCF({"circle_left|circle_right" : sigma}, default = 0)

f = 50 #(Hz)
w = f*2*np.pi

# H1 = ngs.H1(mesh, order = 2, dirichlet = "ambient_face")
H1 = ngs.H1(mesh, order = 3, dirichlet = "ambient_face", complex = True)
u,v = H1.TnT()

K = ngs.BilinearForm(H1)
K += (1/cf_mu*ngs.grad(u)*ngs.grad(v))*ngs.dx + ((1j*w)*sigma*u*v)*ngs.dx("circle_left|circle_right")
K.Assemble()

f = ngs.LinearForm(H1)
f += (cf_Js*v)*ngs.dx()
f.Assemble()


A = ngs.GridFunction(H1)
A.vec.data += K.mat.Inverse(H1.FreeDofs())*f.vec

B = ngs.grad(A)

J = (cf_Js -(1j*w)*cf_sigma*A)
intJ = ngs.Integrate(J, mesh)

# J = J/intJ*I

Draw(ngs.Norm(J), mesh, settings = {"Objects": {"Wireframe": False}})
Draw(ngs.Norm(B),mesh, settings = {"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

In [5]:
ngs.Integrate(cf_Js, mesh)

271.8270213973592

In [22]:
# Parameters

circ_mass = np.pi*0.031**2
circ_number = 58

mu0 = 1.256*1e-6
mu_iron = 1*mu0
sigma = 58*1e6
J = 1e7/(circ_mass)
eps = 1e-8

cf_Js = mesh.MaterialCF({"circle_left": J, "circle_right": -J}, default = 0)
cf_mu = mesh.MaterialCF({"circle_left|circle_right|left|right|bridge" : mu0, "ambient": mu_iron})
cf_sigma = mesh.MaterialCF({"circle_left|circle_right" : sigma}, default = 0)

f = 50 #(Hz)
w = f*2*np.pi

# H1 = ngs.H1(mesh, order = 2, dirichlet = "ambient_face")
H1 = ngs.H1(mesh, order = 3, dirichlet = "ambient_face", complex = True)
PH1 = ngs.H1(mesh, order = 1, dirichlet = "ambient_face", complex = True)

X = H1*PH1

# u,v = X.TnT()
(u,p), (v,q) = X.TnT()

K = ngs.BilinearForm(X)
K += (1/cf_mu*ngs.grad(u)*ngs.grad(v))*ngs.dx +((1j*w)*sigma*u*v)*ngs.dx("circle_left|circle_right")\
    +((1j*w)*sigma*u*q)*ngs.dx("circle_left|circle_right")\
    +((1j*w)*sigma*p*v)*ngs.dx("circle_left|circle_right")\
    + 1/eps*p*q*ngs.dx("circle_left|circle_right")
K.Assemble()

f = ngs.LinearForm(X)
f += (cf_Js*v)*ngs.dx() + ((1-cf_Js)*q)*ngs.dx("circle_left|circle_right")
f.Assemble()


A_and_L = ngs.GridFunction(X)
A_and_L.vec.data += K.mat.Inverse(X.FreeDofs())*f.vec

A = A_and_L.components[0]
L = A_and_L.components[1]


B = ngs.grad(A)

J = (cf_Js -(1j*w)*cf_sigma*A-(1j*w)*cf_sigma*L)

# # J = J/intJ*I

Draw(J, mesh, settings = {"Objects": {"Wireframe": False}})
Draw(ngs.Norm(B),mesh, settings = {"Objects": {"Wireframe": False}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene